In [2]:
import os
from dotenv import load_dotenv

# 1) Cargar .env
load_dotenv()

print("📂 CWD:", os.getcwd())
print("🔧 HOPSWORKS_PROJECT:", os.getenv("HOPSWORKS_PROJECT"))
print("🔧 HOPSWORKS_HOST   :", os.getenv("HOPSWORKS_HOST"))
print("🔧 API_KEY_SET      :", bool(os.getenv("HOPSWORKS_API_KEY")))

# 2) Conexión a Hopsworks con mensajes claros
import hopsworks

project_name = os.getenv("HOPSWORKS_PROJECT")
api_key      = os.getenv("HOPSWORKS_API_KEY")
host         = os.getenv("HOPSWORKS_HOST", "c.app.hopsworks.ai")

if not project_name or not api_key:
    raise RuntimeError("❌ Faltan HOPSWORKS_PROJECT o HOPSWORKS_API_KEY en el entorno/.env")

print("\n🚀 Llamando a hopsworks.login()...")

try:
    project = hopsworks.login(
        api_key_value=api_key,
        project=project_name,
        host=host,
    )
    print(f"✅ Login correcto. Proyecto: {project.name}")

    fs = project.get_feature_store()
    print("✅ Feature Store inicializado.")
except Exception as e:
    print("❌ Error al conectar con Hopsworks:")
    print(repr(e))
    raise



📂 CWD: c:\TEMP\ALOZANO\GRASACORPORAL\notebooks
🔧 HOPSWORKS_PROJECT: GrasaCorporal
🔧 HOPSWORKS_HOST   : c.app.hopsworks.ai
🔧 API_KEY_SET      : True

🚀 Llamando a hopsworks.login()...
2025-11-10 16:33:01,504 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-10 16:33:01,511 INFO: Initializing external client
2025-11-10 16:33:01,512 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-11-10 16:33:02,811 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1234074
✅ Login correcto. Proyecto: GrasaCorporal
✅ Feature Store inicializado.


In [3]:
# Celda 2 — Leer Feature Groups y ver tamaños

from datetime import datetime

# 1) user_fat_percentage (principal)
try:
    fg_main = fs.get_feature_group("user_fat_percentage", version=1)
    df_main = fg_main.read()
    print("✅ user_fat_percentage v1 leído correctamente.")
    print("   Filas:", len(df_main))
    print("   Columnas:", list(df_main.columns))
except Exception as e:
    print("❌ Error leyendo user_fat_percentage v1:")
    print(repr(e))
    df_main = None

print("\n" + "-"*80 + "\n")

# 2) user_fat_feedback (si existe)
try:
    fg_fb = fs.get_feature_group("user_fat_feedback", version=1)
    df_fb = fg_fb.read()
    print("✅ user_fat_feedback v1 leído correctamente.")
    print("   Filas:", len(df_fb))
    print("   Columnas:", list(df_fb.columns))
except Exception as e:
    print("ℹ️ No se pudo leer user_fat_feedback v1 (puede que aún no exista):")
    print(repr(e))
    df_fb = None

# Vista rápida (si hay datos)
if df_main is not None:
    display(df_main.head())

if df_fb is not None and not df_fb.empty:
    display(df_fb.head())


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.86s) 
✅ user_fat_percentage v1 leído correctamente.
   Filas: 11
   Columnas: ['age', 'gender', 'weight_kg', 'height_m', 'max_bpm', 'avg_bpm', 'resting_bpm', 'session_duration_hours', 'calories_burned', 'workout_type', 'water_intake_liters', 'workout_frequency_days_week', 'experience_level', 'bmi', 'log_age', 'email', 'timestamp', 'user_id', 'predicted_fat_percentage', 'real_fat_percentage']

--------------------------------------------------------------------------------

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.76s) 
✅ user_fat_feedback v1 leído correctamente.
   Filas: 4
   Columnas: ['user_id', 'email', 'timestamp', 'real_fat_percentage', 'predicted_fat_percentage']


,age,gender,weight_kg,height_m,max_bpm,avg_bpm,resting_bpm,session_duration_hours,calories_burned,workout_type,water_intake_liters,workout_frequency_days_week,experience_level,bmi,log_age,email,timestamp,user_id,predicted_fat_percentage,real_fat_percentage
0,51,Female,60.00,1.69,120,95,39,1.0,400.0,Cardio,2.0,3,4,21.007668,3.931826,alicia.lozano.marquez@gmail.com,2025-09-20 17:34:07+00:00,1,26.256000,NaN
1,51,Female,59.98,1.69,100,85,39,1.0,350.0,HIIT,2.0,5,4,21.000665,3.931826,alicia.lozano.marquez@gmail.com,2025-09-24 18:46:43+00:00,1,25.693000,NaN
2,51,Female,60.00,1.69,110,87,39,1.2,350.0,Mixed,2.0,5,4,21.007668,3.931826,alicia.lozano.marquez@gmail.com,2025-09-20 17:36:43+00:00,1,25.947000,NaN
3,51,Female,70.00,1.70,180,140,60,1.0,400.0,Cardio,2.0,3,1,24.221453,3.951244,alicia.lozano.marquez@gmail.com,2025-11-03 20:56:45.599297+00:00,1e41a7e106bb8a79,30.221328,NaN
4,51,Female,70.00,1.70,120,90,40,1.0,400.0,Cardio,2.0,5,2,24.221453,3.951244,alicia.lozano.marquez@gmail.com,2025-11-03 21:08:38.048957+00:00,1e41a7e106bb8a79,29.740224,NaN


,user_id,email,timestamp,real_fat_percentage,predicted_fat_percentage
0,1e41a7e106bb8a79,alicia.lozano.marquez@gmail.com,2025-11-05 20:18:30.890935+00:00,25.0,NaN
1,1e41a7e106bb8a79,alicia.lozano.marquez@gmail.com,2025-11-08 12:22:00.791421+00:00,25.0,25.47
2,1e41a7e106bb8a79,alicia.lozano.marquez@gmail.com,2025-11-08 12:38:41.033142+00:00,29.5,29.81
3,1e41a7e106bb8a79,alicia.lozano.marquez@gmail.com,2025-11-09 11:37:56.325362+00:00,29.7,29.81


In [4]:
# Celda 3
import numpy as np

if df_main is None:
    raise RuntimeError("df_main no está disponible. Revisa la Celda 2.")

# Si no hay feedback, avisamos y paramos aquí
if df_fb is None or df_fb.empty:
    print("ℹ️ No hay feedback todavía en user_fat_feedback v1. Nada que monitorizar aún.")
else:
    print("🔗 Uniendo predicciones con feedback...")

    # Aseguramos tipos consistentes
    df_main_ = df_main.copy()
    df_fb_ = df_fb.copy()

    df_main_["user_id"] = df_main_["user_id"].astype(str)
    df_fb_["user_id"] = df_fb_["user_id"].astype(str)

    # Redondeamos timestamps a segundos para mejorar el match
    df_main_["timestamp"] = pd.to_datetime(df_main_["timestamp"]).dt.round("S")
    df_fb_["timestamp"] = pd.to_datetime(df_fb_["timestamp"]).dt.round("S")

    # Join por user_id + email + timestamp
    joined = pd.merge(
        df_main_,
        df_fb_,
        on=["user_id", "email", "timestamp"],
        how="inner",
        suffixes=("_predreg", "_fb"),
    )

    print("📊 Filas unidas (predicción + feedback):", len(joined))

    if joined.empty:
        print("⚠️ No se encontró ninguna fila donde coincidan exactamente user_id + email + timestamp.")
        print("   Esto puede pasar si el feedback se envió en momentos distintos o desde otra fuente.")
    else:
        # Usamos las columnas de feedback como verdad real
        y_true = joined["real_fat_percentage"].astype(float)
        # Preferimos la predicción guardada en feedback si existe, si no la de pred
        y_pred = np.where(
            joined["predicted_fat_percentage_fb"].notna(),
            joined["predicted_fat_percentage_fb"],
            joined["predicted_fat_percentage_predreg"],
        ).astype(float)

        errors = y_pred - y_true
        abs_errors = np.abs(errors)

        mae = abs_errors.mean()
        rmse = np.sqrt((errors**2).mean())
        bias = errors.mean()

        print("\n📈 Métricas de desempeño basadas en feedback real:")
        print(f"   Número de muestras con feedback: {len(joined)}")
        print(f"   MAE  (Error absoluto medio): {mae:.3f} puntos de % grasa")
        print(f"   RMSE (Raíz error cuadrático): {rmse:.3f}")
        print(f"   Bias (Pred - Real)         : {bias:.3f} (positivo = sobreestima)")

        # Tabla rápida por usuario
        resumen_user = (
            joined.assign(abs_error=abs_errors)
                  .groupby("email")
                  .agg(
                      n=("real_fat_percentage", "size"),
                      mae=("abs_error", "mean"),
                      mean_real=("real_fat_percentage", "mean"),
                      mean_pred=("predicted_fat_percentage_predreg", "mean"),
                  )
                  .sort_values("mae", ascending=False)
        )

        print("\n👤 Errores agregados por usuario (top):")
        display(resumen_user.head(10))

        # Guardamos para posibles celdas siguientes
        df_joined = joined


🔗 Uniendo predicciones con feedback...


NameError: name 'pd' is not defined